<a href="https://colab.research.google.com/github/laythamra28/DeepLearning-Project/blob/main/HyperParameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 11.8 MB/s 
     |████████████████████████████████| 209 kB 4.7 MB/s 
     |████████████████████████████████| 81 kB 3.6 MB/s 
     |████████████████████████████████| 78 kB 1.8 MB/s 
     |████████████████████████████████| 147 kB 16.8 MB/s 
     |████████████████████████████████| 50 kB 1.8 MB/s 
     |████████████████████████████████| 112 kB 53.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=bdbe120f2bcaa2be7851b91fabeda5e3c18ebf0b0d84f1cadcc372ea9c70a122
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os
import numpy as np
import torch
import torch.nn as nn
import seaborn as sns
import time

In [ ]:
%cd drive/Shareddrives/'BE 500'/Dataset

/content/drive/Shareddrives/BE 500/Dataset


In [ ]:
%cd ..

/content/drive/Shareddrives/BE 500


In [ ]:
num_classes = 1

class LeNet5(nn.Module):
    def __init__(self, num_classes):
      super().__init__()

      # First layer
      self.conv1 = torch.nn.Conv2d(1,6,kernel_size=5, stride=1, padding=0)
      self.bnorm = torch.nn.BatchNorm2d(6)
      self.relu1 = torch.nn.ReLU()
      self.pool1 = torch.nn.MaxPool2d(kernel_size = 2, stride = 2)

      # Second layer
      self.conv2 = torch.nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
      self.bnorm2 = torch.nn.BatchNorm2d(16)
      self.relu2 = torch.nn.ReLU()
      self.pool2 = torch.nn.MaxPool2d(kernel_size = 2, stride = 2)

      self.flatten=torch.nn.Flatten()

      self.fc = torch.nn.Linear(293904, 400)
      self.relu3 = torch.nn.ReLU()

      self.fc1 = torch.nn.Linear(400, 120)
      self.relu4 = torch.nn.ReLU()

      self.fc2 = torch.nn.Linear(120, 84)
      self.relu5 = torch.nn.ReLU()

      self.fc3 = torch.nn.Linear(84, num_classes)

      self.sigmoid=nn.Sigmoid()
        
    def forward(self, x):

      out = self.conv1(x)
      out = self.relu1(out)
      out = self.pool1(out)
      out = self.conv2(out)
      out = self.relu2(out)
      out = self.pool2(out)
      out = self.flatten(out)
      out = self.fc(out)
      out = self.relu3(out)
      out = self.fc1(out)
      out = self.relu4(out)
      out = self.fc2(out)
      out = self.relu5(out)
      out = self.fc3(out)
      out = self.sigmoid(out)

      return out

In [ ]:
#load in training variables and testing variables
X_train=np.load('X_train.npy')
y_train=np.load('y_train.npy')
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)

X_val = np.load('X_val.npy')
y_val = np.load('y_val.npy')
X_val = X_val.astype(np.float32)
y_val = y_val.astype(np.float32)

X_test=np.load('X_test.npy')
y_test=np.load('y_test.npy')
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

In [ ]:
#Reshape Xtrain
Xtrain=torch.from_numpy(X_train)
Xtrain=Xtrain.reshape((750,1,480,640))

#Reshape Xval
Xval=torch.from_numpy(X_val)
Xval=Xval.reshape((150,1,480,640))

#Reshape Xtest
Xtest=torch.from_numpy(X_test)
Xtest=Xtest.reshape((100,1,480,640))



In [ ]:
ytrain = torch.from_numpy(y_train)
yval = torch.from_numpy(y_val)

In [ ]:
#Use GPU if avaliable
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from torch.utils.data import DataLoader,TensorDataset, random_split

dataset = TensorDataset(Xtrain, ytrain)
trainloader=DataLoader(dataset=dataset,batch_size=64)

dataset_val = TensorDataset(Xval, yval)
valloader=DataLoader(dataset=dataset_val,batch_size=64)

In [ ]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


EPOCHS=30
  
# Train and evaluate the accuarcy of neural network model
def train_and_evaluate(param, model,train_dataloader,val_dataloader):
    
    

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = torch.nn.BCELoss()
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(EPOCHS):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in train_dataloader:

                train_label = train_label.to(device)
                train_input = train_input.to(device)

                output = model(train_input.float())
                
                batch_loss = criterion(output, train_label.float())
                total_loss_train += batch_loss.item()
                
                acc = (output.round() == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    val_input = val_input.to(device)

                    output = model(val_input.float())

                    batch_loss = criterion(output, val_label.float())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.round() == val_label).sum().item()
                    total_acc_val += acc
            
            accuracy = total_acc_val/150

    return accuracy
  
 # Define a set of hyperparameter values, build the model, train the model, and evaluate the accuracy 
def objective(trial):

     params = {
              'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              }
    
     model = LeNet5(1)
    
     accuracy = train_and_evaluate(params, model,trainloader,valloader)

     return accuracy

In [ ]:
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=30)

[I 2022-12-12 02:42:50,043] A new study created in memory with name: no-name-d87b50d8-1d4b-4f4d-aede-2218149fe93c
<ipython-input-11-969d37fb42da>:76: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[I 2022-12-12 02:44:10,521] Trial 0 finished with value: 0.9066666666666666 and parameters: {'learning_rate': 0.0037866193363966377, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.9066666666666666.
[I 2022-12-12 02:45:32,995] Trial 1 finished with value: 0.5133333333333333 and parameters: {'learning_rate': 0.0008675989253230231, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.9066666666666666.
[I 2022-12-12 02:46:57,785] Trial 2 finished with value: 0.9266666666666666 and parameters: {'learning_rate': 0.00015977542670628632, 'optimizer': 'Adam'

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
import matplotlib

In [ ]:
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 
plt=optuna.visualization.plot_param_importances(study)
plt.xtick

In [ ]:
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

learning_rate: 3.795899556767346e-05
optimizer: Adam


In [ ]:
!jupyter nbconvert --to html "HyperParameter-Tuning.ipynb"

[NbConvertApp] Converting notebook HyperParameter-Tuning.ipynb to html
[NbConvertApp] Writing 342042 bytes to HyperParameter-Tuning.html
